In [1]:
import requests
from bs4 import BeautifulSoup as BS
import re
import codecs
import pandas as pd

## 定义functions

需要的function：
- get url list
- get info
    - spider1 + spider2 + spider3
    - 试试能不能写在一起
- 变成pandas表格，清理DataFrame

### get url list

In [18]:
mid = '742470'
def get_url_list(mid):
    base_url = 'http://space.bilibili.com/ajax/member/getSubmitVideos?page='
    url_list = []
    no_page = int(re.findall('pages":(\d+)', str(requests.get('http://space.bilibili.com/ajax/member/getSubmitVideos?page=1&mid=742470').content))[0])
    for i in range(1, no_page+1):
        url_list.append(base_url + '%d&mid=%s' % (i,mid))
    return url_list

In [ ]:
'http://space.bilibili.com/ajax/member/getSubmitVideos?page=1&mid=742470'

In [8]:
re.findall('pages":(\d+)', str(requests.get('http://space.bilibili.com/ajax/member/getSubmitVideos?page=1&mid=742470').content))[0]

'8'

In [11]:
mid = '742470'
type(mid)

str